# Homework

This homework asks you to compute a function's derivatives using symbolic and automatic differentiation methods. Our objective function is the follows.

\begin{align}
 f(x) = \log(x)\times \exp\left[-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2 \right].
\end{align}

Let's assume $\mu=1.5$ and $\sigma=2$. 



## Automatic Differentiations

### Use the Taylor expansion to derive the result of logarithms on dual numbers.

Let $f(x)=log(x)$ and so $f'(x)=\frac 1x$. For a dual number, it means $f(a+b\epsilon)=log(a+b\epsilon)$. Applying the Taylor expansion, we have

\begin{align}
 f(a +b\epsilon) & = f(a) + f'(a)b\epsilon = log(a) + \frac 1a b\epsilon; \\
\Longrightarrow \quad log(a+b\epsilon) & = log(a) + \frac 1a b\epsilon.
\end{align}

Again, the first "=" is according to the Taylor expansion of $f(x)$, and the 2nd "=" follows from the definition of $f(x)$ and $f'(x)$.

### Add methods of the division ("/") and logarithm ("log()") for dual numbers in Julia.

In [1]:
import Base: +, *, -, ^, exp, log, / # 增加自定義的運算

struct DualNumber{T1, T2} <: Real   
    re::T1                # differnt types so (Float64, Int64), etc., is possible
    du::T2
end

+(x::DualNumber, y::DualNumber) = DualNumber(x.re + y.re, x.du + y.du)  # dual addition
+(x::DualNumber, a::Number) = DualNumber(x.re + a, x.du)  
+(a::Number, x::DualNumber) = DualNumber(x.re + a, x.du) 

-(x::DualNumber, y::DualNumber) = DualNumber(x.re - y.re, x.du - y.du)  # dual subtraction
-(x::DualNumber, a::Number) = DualNumber(x.re - a, x.du)  
-(a::Number, x::DualNumber) = DualNumber(a-x.re, -x.du) 

*(x::DualNumber, y::DualNumber) = DualNumber(x.re*y.re, x.re*y.du + y.re*x.du)
*(x::DualNumber, a::Number) = DualNumber(a*x.re, a*x.du)
*(a::Number, x::DualNumber) = DualNumber(a*x.re, a*x.du)

# Where is the division rule? Your turn!

/(x::DualNumber, y::DualNumber) = DualNumber(x.re / y.re, x.du * y.re - x.re * y.du / y.re^2)
/(x::DualNumber, a::Number) = DualNumber(x.re / a, x.du / a)
/(a::Number, x::DualNumber) = DualNumber(a / x.re, -a * x.du / x.re^2)

^(x::DualNumber, n::Union{Float64, Int64}) = DualNumber(x.re^n, n*x.re^(n-1)*x.du)  # cannot use n<:Real, since n is variable

exp(x::DualNumber) = DualNumber(exp(x.re), exp(x.re)*x.du)

# Where is the log rule? Your turn!

log(x::DualNumber) = DualNumber(log(x.re), x.du/x.re) 

log (generic function with 24 methods)

### Use the dual numbers you've defined to compute the derivative of $f(x)$ at $x=1.2$.

In [2]:
using Printf 
Base.show(io::IO, f::Float64) = @printf(io, "%.16f", f)

μ=1.5
σ=2
x0 = 1.2

f(x) =log(x)*exp(-0.5*((x-μ)/σ)^2)
f_d1_true(x) =(1/x)*exp(-0.5*((x-μ)/σ)^2)+log(x)*exp(-0.5*((x-μ)/σ)^2)*(-(x-μ)/σ)*(1/σ)

x_tilde = DualNumber(x0, 1.0) # x_tilde =x0+\epsilon 
mydf = f(x_tilde)
error = mydf.du-f_d1_true(x0)

@show mydf.du   # result from dual number
@show f_d1_true(x0)   # result from analytic equation 
@show error

mydf.du = 0.8375320155347377
f_d1_true(x0) = 0.8375320155347377
error = 0.0000000000000000


0.0000000000000000

### Use Julia's package `ForwardDiff`, which implements the forward-mode auto differentiation, to compute the derivative of $f(x)$ at $x=1.2$.

In [3]:
using Printf 
Base.show(io::IO, f::Float64) = @printf(io, "%.16f", f)

μ=1.5
σ=2
x0 = 1.2

f(x) =log(x)*exp(-0.5*((x-μ)/σ)^2)
f_d1_true(x) =(1/x)*exp(-0.5*((x-μ)/σ)^2)+log(x)*exp(-0.5*((x-μ)/σ)^2)*(-(x-μ)/σ)*(1/σ)

using ForwardDiff

julia_AD_error = ForwardDiff.derivative(f, x0) - f_d1_true(x0)  # automatic differentiation gives exact answer
@show ForwardDiff.derivative(f, x0)
@show julia_AD_error

ForwardDiff.derivative(f, x0) = 0.8375320155347377
julia_AD_error = 0.0000000000000000


[ Info: Precompiling ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]


0.0000000000000000

## Symbolic Differentiation

### Use Julia's package `SymPy` to compute the analytic derivative of $f(x)$ and evaluate the result at $x=1.2$.
- Hint: You may need to use `diff` and `subs` functions in the package.
- `diff()` function computes the symbolic derivative of a given expression with respect to a specified variable. The syntax for `diff()` function is `diff(expr, var)`, where `expr` is the expression to be differentiated and `var` is the variable with respect to which the derivative is to be taken.
- `subs()` function, on the other hand, substitutes a value or expression for a variable in a given expression. The syntax for `subs()` function is `subs(expr, old, new)`, where `expr` is the expression to be substituted, `old` is the variable or expression to be replaced, and `new` is the replacement value or expression. 
- Remark: I intended to use `Symbolics` but find it less straightforward. You might give it a try.

In [4]:
using Symbolics

@variables x


f(x) =log(x)*exp(-0.5*((x-μ)/σ)^2)

Symbolics.derivative(f(x),x) |> display

[ Info: Precompiling Symbolics [0c5d862f-8b57-4792-8d23-62f2024744c7]


exp(-0.1250000000000000((x - 1.5000000000000000)^2)) / x - 0.2500000000000000(x - 1.5000000000000000)*log(x)*exp(-0.1250000000000000((x - 1.5000000000000000)^2))

In [14]:
using SymPy
x, μ, σ=symbols("x,μ,σ")



f(x) =log(x)*exp(-0.5*((x-μ)/σ)^2)
f_d1_true(x) =(1/x)*exp(-0.5*((x-μ)/σ)^2)+log(x)*exp(-0.5*((x-μ)/σ)^2)*(-(x-μ)/σ)*(1/σ)

f_d1_true(x) |> display

                      2                        2 
          -0.5⋅(x - μ)             -0.5⋅(x - μ)  
          ──────────────           ──────────────
                 2                        2      
                σ                        σ       
(-x + μ)⋅ℯ              ⋅log(x)   ℯ              
─────────────────────────────── + ───────────────
                2                        x       
               σ                                 

In [19]:
using SymPy

μ = 1.5
σ = 2
x0 = 1.2

@vars x
f(x) = log(x) * exp(-0.5 * ((x - μ) / σ)^2)

df = diff(f(x), x)

println("The symbolic derivative of f is: ", df)

df_x0 = subs(df, x, x0)

println("The symbolic derivative of f at x = 1.2 is: ", df_x0)

error = f_d1_true(x0) - df_x0
@show error


The symbolic derivative of f is: (0.375 - 0.25*x)*exp(-0.28125*(0.666666666666667*x - 1)^2)*log(x) + exp(-0.28125*(0.666666666666667*x - 1)^2)/x
The symbolic derivative of f at x = 1.2 is: 0.837532015534738
error = 0


0